## Import Modules/

In [2]:
import pandas
import requests

## Import Data 

In [4]:
data = pandas.read_excel("/users/danielcorcoran/desktop/mobile_speed_camera_data/Mobile Camera Locations August 2018(1).xlsx")

In [5]:
data.head(3)

,MOBILE SAFETY CAMERA LOCATION DESCRIPTION REPORT,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,LOCATION,SUBURB,Reason Code,Audit Date
1,Agar Road,CORONET BAY,BC,Jul-2018
2,Aitken Boulevard,CRAIGIEBURN,ABC,Dec-2017


## Manipulate Data 

In [6]:
headers = data.loc[0].tolist()
headers

['LOCATION', 'SUBURB', 'Reason Code', 'Audit Date']

In [7]:
data.columns = headers

In [8]:
data.head()

,LOCATION,SUBURB,Reason Code,Audit Date
0,LOCATION,SUBURB,Reason Code,Audit Date
1,Agar Road,CORONET BAY,BC,Jul-2018
2,Aitken Boulevard,CRAIGIEBURN,ABC,Dec-2017
3,Aitken Boulevard,ROXBURGH PARK,ABC,Dec-2017
4,Aitken Street,ALEXANDRA,C,Apr-2018


In [9]:
data.drop(0, axis = 0, inplace = True)

In [10]:
data.head()

,LOCATION,SUBURB,Reason Code,Audit Date
1,Agar Road,CORONET BAY,BC,Jul-2018
2,Aitken Boulevard,CRAIGIEBURN,ABC,Dec-2017
3,Aitken Boulevard,ROXBURGH PARK,ABC,Dec-2017
4,Aitken Street,ALEXANDRA,C,Apr-2018
5,Albert Avenue,BORONIA,C,Jan-2018


In [16]:
data.reset_index(drop = True, inplace = True)

In [17]:
data.shape

(1677, 5)

In [18]:
data["full_location"] = data["LOCATION"] +  ", " + data["SUBURB"] + ", VICTORIA, AUSTRALIA"

In [19]:
data.head()

,LOCATION,SUBURB,Reason Code,Audit Date,full_location
0,Agar Road,CORONET BAY,BC,Jul-2018,"Agar Road, CORONET BAY, VICTORIA, AUSTRALIA"
1,Aitken Boulevard,CRAIGIEBURN,ABC,Dec-2017,"Aitken Boulevard, CRAIGIEBURN, VICTORIA, AUSTR..."
2,Aitken Boulevard,ROXBURGH PARK,ABC,Dec-2017,"Aitken Boulevard, ROXBURGH PARK, VICTORIA, AUS..."
3,Aitken Street,ALEXANDRA,C,Apr-2018,"Aitken Street, ALEXANDRA, VICTORIA, AUSTRALIA"
4,Albert Avenue,BORONIA,C,Jan-2018,"Albert Avenue, BORONIA, VICTORIA, AUSTRALIA"


In [20]:
data.to_csv("aug_mobile_cameras.csv", index_label="ROW_INDEX")

## Geocoding Function

In [21]:
def return_address_dictionary(address):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?address="
    full_url = base_url + address
    request = requests.get(full_url)
    result = request.json()
    
    longitude = result["results"][0]["geometry"]["location"]["lng"]
    latitude = result["results"][0]["geometry"]["location"]["lat"]
    formatted_address = result["results"][0]["formatted_address"]

    return longitude, latitude, formatted_address

In [22]:
for index in range(data.shape[0]): #Adjust index ranges upon failures
    
    address = data.loc[index, "full_location"]
    long, lat, address = return_address_dictionary(address)
    
    print(index, long, lat, address, sep = "|")

SSLError: HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=Agar%20Road,%20CORONET%20BAY,%20VICTORIA,%20AUSTRALIA (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:833)'),))

In [54]:
data.head()

,LOCATION,SUBURB,Reason Code,Audit Date,full_location
0,Agar Road,CORONET BAY,BC,Jul-2018,"Agar Road, CORONET BAY"
1,Aitken Boulevard,CRAIGIEBURN,ABC,Dec-2017,"Aitken Boulevard, CRAIGIEBURN"
2,Aitken Boulevard,ROXBURGH PARK,ABC,Dec-2017,"Aitken Boulevard, ROXBURGH PARK"
3,Aitken Street,ALEXANDRA,C,Apr-2018,"Aitken Street, ALEXANDRA"
4,Albert Avenue,BORONIA,C,Jan-2018,"Albert Avenue, BORONIA"


In [55]:
data["row_index"] = data.index

In [57]:
data.head()

,LOCATION,SUBURB,Reason Code,Audit Date,full_location,row_index
0,Agar Road,CORONET BAY,BC,Jul-2018,"Agar Road, CORONET BAY",0
1,Aitken Boulevard,CRAIGIEBURN,ABC,Dec-2017,"Aitken Boulevard, CRAIGIEBURN",1
2,Aitken Boulevard,ROXBURGH PARK,ABC,Dec-2017,"Aitken Boulevard, ROXBURGH PARK",2
3,Aitken Street,ALEXANDRA,C,Apr-2018,"Aitken Street, ALEXANDRA",3
4,Albert Avenue,BORONIA,C,Jan-2018,"Albert Avenue, BORONIA",4


## Bring in results data (excel sheet created from geocoding results)

In [58]:
results_data = pandas.read_excel("/users/danielcorcoran/desktop/geocode_results.xlsx")

In [60]:
results_data.head()

,index,longitude,latitude,corrected address
0,0.0,145.446520,-38.435757,"Agar Rd, Coronet Bay VIC 3984, Australia"
1,1.0,144.909917,-37.594783,"Aitken Blvd, Craigieburn VIC 3064, Australia"
2,2.0,144.914952,-37.624340,"Aitken Blvd, Roxburgh Park VIC 3064, Australia"
3,3.0,145.703606,-37.187826,"Aitken St, Alexandra VIC 3714, Australia"
4,4.0,145.297632,-37.854281,"Albert Ave, Boronia VIC 3155, Australia"


## Merge to original 

In [74]:
merged_data = pandas.merge(left = data,
                          right = results_data,
                          how = "left",
                          left_on = "row_index",
                          right_on = "index")

In [75]:
merged_data["extracted_on"] = "2018-August-06"

In [76]:
merged_data.to_csv("geocoded_mobile_speed_cameras_august_6_2018.csv", sep = ",", index = False)

## Bring merged data back in

In [25]:
merged_data = pandas.read_csv("/users/danielcorcoran/desktop/mobile_speed_camera_data/geocoded_mobile_speed_cameras_august_6_2018.csv")

In [36]:
merged_data.loc[0,"longitude"]

145.4465196

In [26]:
merged_data

,LOCATION,SUBURB,Reason Code,Audit Date,full_location,row_index,index,longitude,latitude,corrected address,extracted_on
0,Agar Road,CORONET BAY,BC,Jul-2018,"Agar Road, CORONET BAY",0,0.0,145.446520,-38.435757,"Agar Rd, Coronet Bay VIC 3984, Australia",2018-August-06
1,Aitken Boulevard,CRAIGIEBURN,ABC,Dec-2017,"Aitken Boulevard, CRAIGIEBURN",1,1.0,144.909917,-37.594783,"Aitken Blvd, Craigieburn VIC 3064, Australia",2018-August-06
2,Aitken Boulevard,ROXBURGH PARK,ABC,Dec-2017,"Aitken Boulevard, ROXBURGH PARK",2,2.0,144.914952,-37.624340,"Aitken Blvd, Roxburgh Park VIC 3064, Australia",2018-August-06
3,Aitken Street,ALEXANDRA,C,Apr-2018,"Aitken Street, ALEXANDRA",3,3.0,145.703606,-37.187826,"Aitken St, Alexandra VIC 3714, Australia",2018-August-06
4,Albert Avenue,BORONIA,C,Jan-2018,"Albert Avenue, BORONIA",4,4.0,145.297632,-37.854281,"Albert Ave, Boronia VIC 3155, Australia",2018-August-06
5,Albert Road,WARRAGUL,ABC,Apr-2018,"Albert Road, WARRAGUL",5,5.0,145.948668,-38.162433,"Albert Rd, Warragul VIC 3820, Australia",2018-August-06
6,Alexanders Road,MORWELL,ABC,Apr-2018,"Alexanders Road, MORWELL",6,6.0,146.444067,-38.218357,"Alexanders Rd, Morwell VIC 3840, Australia",2018-August-06
7,Allan Street,KYABRAM,C,Apr-2018,"Allan Street, KYABRAM",7,7.0,145.049871,-36.313490,"Allan St, Kyabram VIC 3620, Australia",2018-August-06
8,Allenby Avenue,RESERVOIR,C,Apr-2018,"Allenby Avenue, RESERVOIR",8,8.0,145.006307,-37.702616,"Allenby Ave, Reservoir VIC 3073, Australia",2018-August-06
9,Allies Road,MYERS FLAT,AC,Jul-2018,"Allies Road, MYERS FLAT",9,9.0,144.200895,-36.720110,"Allies Rd, Myers Flat VIC 3556, Australia",2018-August-06


In [32]:
merged_data.columns

Index(['LOCATION', 'SUBURB', 'Reason Code', 'Audit Date', 'full_location',
       'row_index', 'index', 'longitude', 'latitude', 'corrected address',
       'extracted_on'],
      dtype='object')

## Correcting isssue with addresses outside of victoria

In [60]:
victoria_master_bounds = [140.96168199600004,
 -39.159189527999956,
 149.9762909950001,
 -33.98042558699996]

for index in range(merged_data.shape[0]):
    long = merged_data.loc[index, 'longitude']
    lat = merged_data.loc[index, 'latitude']
    
    if long >= victoria_master_bounds[0] and long <= victoria_master_bounds[2] and lat >= victoria_master_bounds[1] and lat <= victoria_master_bounds[3]:
        merged_data.loc[index, "inside_victoria_bounds"] = "True"
    else:
        merged_data.loc[index, "inside_victoria_bounds"] = "False"
        long,lat,formatted_address = return_address_dictionary(merged_data.loc[index, "full_location"] + ", VICTORIA, AUSTRALIA")
        print(long, lat,formatted_address)
        merged_data.loc[index, "longitude"] = long 
        merged_data.loc[index, "latitude"] = lat
        merged_data.loc[index, "corrected_address"] = formatted_address
    

142.0146467 -37.7391806 Burns St, Hamilton VIC 3300, Australia
144.5724233 -38.1648882 Clifton Springs Rd, Clifton Springs VIC 3222, Australia
141.5973179 -38.3319956 Garden St, Portland VIC 3305, Australia
142.0343551 -37.7455669 George St, Hamilton VIC 3300, Australia
142.1886542 -36.7405239 Golf Course Rd, Haven VIC 3401, Australia
142.0211593 -37.7314935 King St, Hamilton VIC 3300, Australia
141.6011746 -38.329709 New St, Portland VIC 3305, Australia
144.6949226 -37.7018025 Plumpton Road, Plumpton VIC 3335, Australia
143.1473452 -38.2329527 Princes Hwy, Camperdown VIC 3260, Australia
145.0937179 -37.8936073 Princes Hwy, Victoria, Australia
145.1415088 -38.1092354 Seaford Rd, Seaford VIC 3198, Australia
144.8091884 -37.7565399 St Albans Rd, St Albans VIC 3021, Australia
144.7274965 -37.590355 The Avenue, Sunbury VIC 3429, Australia
142.02475 -37.7333986 Victoria St, Hamilton VIC 3300, Australia
144.7401475 -36.357462 Webb Rd, Rochester VIC 3561, Australia


In [62]:
merged_data.head()

,LOCATION,SUBURB,Reason Code,Audit Date,full_location,row_index,index,longitude,latitude,corrected address,extracted_on,inside_victoria_bounds,corrected_address
0,Agar Road,CORONET BAY,BC,Jul-2018,"Agar Road, CORONET BAY",0,0.0,145.446520,-38.435757,"Agar Rd, Coronet Bay VIC 3984, Australia",2018-August-06,True,NaN
1,Aitken Boulevard,CRAIGIEBURN,ABC,Dec-2017,"Aitken Boulevard, CRAIGIEBURN",1,1.0,144.909917,-37.594783,"Aitken Blvd, Craigieburn VIC 3064, Australia",2018-August-06,True,NaN
2,Aitken Boulevard,ROXBURGH PARK,ABC,Dec-2017,"Aitken Boulevard, ROXBURGH PARK",2,2.0,144.914952,-37.624340,"Aitken Blvd, Roxburgh Park VIC 3064, Australia",2018-August-06,True,NaN
3,Aitken Street,ALEXANDRA,C,Apr-2018,"Aitken Street, ALEXANDRA",3,3.0,145.703606,-37.187826,"Aitken St, Alexandra VIC 3714, Australia",2018-August-06,True,NaN
4,Albert Avenue,BORONIA,C,Jan-2018,"Albert Avenue, BORONIA",4,4.0,145.297632,-37.854281,"Albert Ave, Boronia VIC 3155, Australia",2018-August-06,True,NaN


In [63]:
merged_data["inside_victoria_bounds"].value_counts()

True    1677
Name: inside_victoria_bounds, dtype: int64

In [64]:
merged_data.to_csv("geocoded_mobile_speed_cameras_august_6_2018_final.csv", index = False)